In [58]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import h5py
import pandas as pd
import numpy as np
from torchvision import models, transforms

In [59]:
# Define  PyTorch model
class PretrainedECGModel(nn.Module):
    def __init__(self, num_classes):
        super(PretrainedECGModel, self).__init__()
        # Load a suitable pretrained ResNet model
        self.pretrained_model = models.resnet18(pretrained=True)
        # Modify the classifier layer of the base model to output the number of classes
        in_features = self.pretrained_model.fc.in_features
        self.pretrained_model.fc = nn.Linear(in_features, num_classes)
        
    def forward(self, x):
        return self.pretrained_model(x)

In [61]:
# Define custom dataset class
class SleepDataset(Dataset):
    def __init__(self, h5_file, label_file, transform=None):
        self.data = self.load_data_from_h5(h5_file)
        self.labels = self.load_labels_from_csv(label_file)
        self.transform = transform
        
        # Check if lengths match
        if len(self.data) != len(self.labels):
            raise ValueError(f"Lengths of data and labels do not match.: {len(self.data)} not equal to {len(self.labels)}")
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample, label
    
    def load_data_from_h5(self, h5_file):
        with h5py.File(h5_file, 'r') as f:
            data = f['signals'][:]
        return data
    
    def load_labels_from_csv(self, label_file):
        labels_df = pd.read_csv(label_file)
        labels = labels_df['label'].values
        return labels

In [62]:
# Function to train the model
def train_model(model, train_loader, optimizer, criterion, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

In [63]:
# Define paths to data and labels
h5_file = 'data/results/MESA/psg_signals/0056.h5'
label_file = 'data/results/MESA/psg_labels/0056.csv'

In [35]:
len(h5_file)

37

In [64]:
# Define transformations to be applied to input data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Use ImageNet normalization
])

In [65]:
# Initialize dataset and dataloader
dataset = SleepDataset(h5_file, label_file, transform=transform)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

ValueError: Lengths of data and labels do not match.: 11058944 not equal to 1440

In [47]:
# Initialize the model
num_classes = 4  # there are 4 sleep stages extracted
model = PretrainedECGModel(num_classes)

/Users/honesty/miniforge3/envs/myenv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/honesty/miniforge3/envs/myenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [48]:
# Define optimizer and criterion
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [66]:
# Train the model
train_model(model, loader, optimizer, criterion, num_epochs=10)
